In [ ]:
import pandas as pd
import logging

# Log fayl manzili
log_path = r"C:\Users\Rasulbek907\Desktop\Final_Project\Log\tuning.log"

# Log sozlamalari
logging.basicConfig(
    filename=log_path,
    filemode='a',  # Append mode
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.INFO
)

try:
    logging.info("CSV fayl o'qilmoqda:...")
    df = pd.read_csv(r"C:\Users\Rasulbek907\Desktop\Final_Project\Data\Feature_selection\Filtered_Features.csv")
    logging.info(f"Fayl muvaffaqiyatli o'qildi. Satırlar soni: {len(df)} ustunlar soni: {len(df.columns)}")
except Exception as e:
    logging.error(f"CSV faylni o'qishda xatolik: {e}")

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor

# X va y ajratamiz
X = df.drop('life_span', axis=1)
y = df['life_span']

# Train / Validation / Test (60/20/20 bo‘linadi)
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)

# Decision Tree Regressor model
dt = DecisionTreeRegressor(random_state=42)
dt.fit(X_train, y_train)

# Baholash (Validation)
y_pred = dt.predict(X_val)

# Natijalar
print("📊 Decision Tree Validation R2:", r2_score(y_val, y_pred))
print("📉 Validation MSE:", mean_squared_error(y_val, y_pred))


In [ ]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score

dt = DecisionTreeRegressor(random_state=42)

# Parametrlar to'plami (tuning uchun)
param_grid = {
    'max_depth': [3, 5, 7, 10, 15, 20, None],              # Daraxt chuqurligi, None — chegarasiz
    'min_samples_split': [2, 5, 10, 15, 20],              # Node bo‘linishi uchun minimal namuna soni
    'min_samples_leaf': [1, 2, 4, 6, 8],                   # Leaf node dagi minimal namuna soni
    'max_features': [None, 'auto', 'sqrt', 'log2'],        # Xususiyatlar soni (None — barcha ustunlar)
    'max_leaf_nodes': [None, 10, 20, 30, 40, 50],          # Maksimal barglar soni
    'min_weight_fraction_leaf': [0.0, 0.01, 0.05],        # Minimal vaznli fraction leaf uchun
    'splitter': ['best', 'random']                         # Qanday qilib split qilish: eng yaxshisi yoki random
}


# GridSearchCV ni sozlash (5-fold cross-validation)
grid_search = GridSearchCV(estimator=dt, param_grid=param_grid,
                           cv=5, scoring='r2', n_jobs=-1, verbose=1)

# Modellarni o'qitish va eng yaxshi parametrlarni topish
grid_search.fit(X_train, y_train)

# Eng yaxshi parametrlar
print("Eng yaxshi parametrlar:", grid_search.best_params_)

# Eng yaxshi model bilan test to'plamida baholash
best_dt = grid_search.best_estimator_
y_pred = best_dt.predict(X_test)

print("Test R2:", r2_score(y_test, y_pred))
print("Test MSE:", mean_squared_error(y_test, y_pred))


In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# Parametrlar diapazoni
param_dist = {
    'max_depth': [3, 5, 7, 10, 15, 20, None],
    'min_samples_split': [2, 5, 10, 15, 20],
    'min_samples_leaf': [1, 2, 4, 6, 8],
    'max_features': [None, 'auto', 'sqrt', 'log2'],
    'max_leaf_nodes': [None, 10, 20, 30, 40, 50],
    'min_weight_fraction_leaf': [0.0, 0.01, 0.05],
    'splitter': ['best', 'random']
}

# Model
dt = DecisionTreeRegressor(random_state=42)

# RandomizedSearchCV obyekti
random_search = RandomizedSearchCV(
    estimator=dt,
    param_distributions=param_dist,
    n_iter=50,              # Necha ta tasodifiy kombinatsiya sinab ko‘riladi
    scoring='r2',
    cv=5,
    verbose=2,
    random_state=42,
    n_jobs=-1               # Paralel ishlatish, barcha yadrodan foydalanish
)

# Modelni o'qitish
random_search.fit(X_train, y_train)

# Eng yaxshi natijalar
print("Best parameters found: ", random_search.best_params_)
print("Best cross-validation R2 score: ", random_search.best_score_)